In [5]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(15):
    # Directory containing your event files
    log_dir = f"/home/<User>/repos/csg_is/fedstellar/app/logs/fedstellar_DFL_03_07_2024_15_03_31/metrics/participant_{i}"
    date_str = "02/07/2024 10:22:20" # minus two hours for timezone

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step  value                     timestamp                    tag
0  2413  100.0 2024-07-03 13:12:23.308784896  Resources/CPU_percent
1  2421   27.4 2024-07-03 13:12:31.382390784  Resources/CPU_percent
2  2429    5.4 2024-07-03 13:12:39.438965760  Resources/CPU_percent
3  2437    5.1 2024-07-03 13:12:47.560524288  Resources/CPU_percent
4  2445    4.6 2024-07-03 13:12:55.647530240  Resources/CPU_percent
2024-07-03 13:16:01 2024-07-03 13:04:37
                step     value                     timestamp
tag                                                         
Test/Accuracy   5138  0.931908 2024-07-03 13:15:55.822451712
Test/F1Score    5138  0.921710 2024-07-03 13:15:55.822494976
Test/Loss       5138  0.087146 2024-07-03 13:15:55.822416128
Test/Precision  5138  0.922744 2024-07-03 13:15:55.822467328
Test/Recall     5138  0.931908 2024-07-03 13:15:55.822481408
   step  value                     timestamp                    tag
0   325  100.0 2024-07-03 13:07:06.756242432  Resources/C

In [6]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,5138,0.931908,2024-07-03 13:15:55.822451712,participant_0
Test/F1Score,5138,0.921710,2024-07-03 13:15:55.822494976,participant_0
Test/Loss,5138,0.087146,2024-07-03 13:15:55.822416128,participant_0
Test/Precision,5138,0.922744,2024-07-03 13:15:55.822467328,participant_0
Test/Recall,5138,0.931908,2024-07-03 13:15:55.822481408,participant_0
...,...,...,...,...
Test/Accuracy,1934,0.928166,2024-07-03 13:12:24.632961536,participant_14
Test/F1Score,1934,0.922939,2024-07-03 13:12:24.633048832,participant_14
Test/Loss,1934,0.119379,2024-07-03 13:12:24.632876288,participant_14


In [9]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_360787/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,2204.2667,0.9164
Test/F1Score,2204.2667,0.9084
Test/Loss,2204.2667,0.1510
Test/Precision,2204.2667,0.9166
Test/Recall,2204.2667,0.9164


In [8]:
concat_df.to_csv("results_csvs/dfl_15_star.csv")